# Trabalho 7
## Rede Madaline para reconhecimento de dígitos
<img src="digitos.png" alt="digitos" width="250"/>

In [116]:
import os
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import random
from collections import defaultdict

from ipywidgets import *
import ipywidgets as widgets

from IPython.display import display, HTML

########## entradas dos parametros ##########
print('\n\nTaxa de aprendizagem:')
def f(taxa_aprendizagem=0.5):
    return taxa_aprendizagem
taxa_aprendizagem = interactive(f, taxa_aprendizagem=(0.001, 1, 0.001))
display(taxa_aprendizagem)

print('\n\nCritério de parada:')
def f(parada):
    return parada
parada = interactive(f, parada=['Erro', 'Ciclos'])
display(parada)

print('\n\nNúmero de ciclos:')
def f(ciclos=10):
    return ciclos
ciclos = interactive(f, ciclos=(1, 50, 1))
display(ciclos)

print('\n\nErro tolerado:')
def f(erro=0.1):
    return erro
erro = interactive(f, erro=(0.001, 100, 0.001))
display(erro)
##############################################

##### conversao das letras em inputs.csv #####
def inputs_to_csv():
    #os.getcwd()
    path = 'trab7/letras'
    files = [f for f in os.listdir(path) if not f.startswith('.')]
    files.sort()
    letters = []
    for file_name in files:
        file = pd.read_csv("{}/{}".format(path, file_name), delim_whitespace=True, header=None)
        letters.append(file.values.flatten())

    row_names = [f[0:2] for f in files]
    pd.DataFrame(np.array(letters)).to_csv("trab7/inputs.csv", header=None, index=None) 
##############################################

class Madaline():
    def __init__(self, conjunto_entradas, targets, taxa_aprendizagem, parada, ciclos_max, 
                 erro_tolerado, classes, **kwargs):
        
        self.numero_entradas = len(conjunto_entradas[0])
        self.conjunto_entradas = np.array(conjunto_entradas)
        self.targets = np.array(targets)
        self.classes = np.array(classes)
        self.taxa_aprendizagem = taxa_aprendizagem
        self.ciclos = 0
        self.ciclos_max = ciclos_max
        self.parada = parada
        self.erro_tolerado = erro_tolerado
        self.pesos = self.__inicializa_pesos()
        self.bias = np.array([random.uniform(-0.5,0.5) for saida in targets[0]])
        self.erro = 0
        
    def __inicializa_pesos(self):
        pesos = []
        for i in range(self.numero_entradas):
            pesos_entrada = []
            for j in range(len(self.targets[0])):
                pesos_entrada.append(random.uniform(-0.5,0.5))
            pesos.append(pesos_entrada)
        return np.array(pesos)
    
    def __f_ativacao(self, yin):
        out = np.copy(yin)
        out[out >= 0] = int(1)
        out[out < 0] = int(0)
        return out
    
    def treinamento(self):
        # x = [21,63], w = [63,7], b = [1,7], t = [21,7], y =x*w = [21,7]
        erros = []
        treinada = False
        self.ciclos = 1;
        while not treinada:
            self.erro = 0   
            yin = self.conjunto_entradas @ self.pesos + self.bias
            y = self.__f_ativacao(yin)
            self.erro = 0.5 * np.sum(np.power((self.targets - y), 2))
            # self.erro = self.erro_tolerado * np.sum(np.power((self.targets - y), 2))
            self.pesos = self.pesos + (self.taxa_aprendizagem * ((self.targets - y).T @ self.conjunto_entradas)).T
            self.bias = self.bias + (self.taxa_aprendizagem * np.sum((self.targets - y), axis=0))
            
            erros.append(self.erro)
            if (self.parada == 'ciclos' and self.ciclos >= self.ciclos_max) \
                or (self.parada == 'erro' and self.erro <= self.erro_tolerado):
                treinada = True
            else:
                self.ciclos += 1
        return list(range(1, self.ciclos+1)), erros
    
    def operacao(self, letra):
        yin = letra @ self.pesos + self.bias
        y = self.__f_ativacao(yin)
        return y, self.classes[y == 1][0]

def processar():
    #display(os.getcwd())
    conjunto_entradas = pd.read_csv('inputs.csv', header=None).values
    targets_dataframe = pd.read_csv('targets.csv')
    targets = targets_dataframe.values
    classes = list(targets_dataframe.columns)
    
    madaline = Madaline(
        conjunto_entradas = conjunto_entradas, 
        targets = targets, 
        taxa_aprendizagem = taxa_aprendizagem.result,
        parada = parada.result.lower(),
        ciclos_max = ciclos.result,
        erro_tolerado = erro.result,
        classes = classes
    )
    
    lista_ciclos, lista_erros = madaline.treinamento()
    
    plt.rcParams["figure.figsize"] = [10,6]
    plt.plot(lista_ciclos, lista_erros, 'r', label="Erro", marker='o', scalex=True)
    plt.style.use('ggplot')
    plt.title('Erro x Ciclo')
    plt.xlabel('Ciclo')
    plt.ylabel('Erro')
    for x,y in zip(lista_ciclos, lista_erros):
        plt.annotate(s='{:3.2f}'.format(y), xy=(x+0.1,y+0.1))
    plt.legend()
    plt.show()
    
    res_classes, classe = madaline.operacao(conjunto_entradas[6])
    #display(res_classes)
    
    
widgets.interact_manual.opts['manual_name'] = 'Processar algoritmo' # muda texto do botao
madaline = interact_manual(processar); # metodo a executar quando pressionar o botao



Taxa de aprendizagem:


interactive(children=(FloatSlider(value=0.5, description='taxa_aprendizagem', max=1.0, min=0.001, step=0.001),…



Critério de parada:


interactive(children=(Dropdown(description='parada', options=('Erro', 'Ciclos'), value='Erro'), Output()), _do…



Número de ciclos:


interactive(children=(IntSlider(value=10, description='ciclos', max=50, min=1), Output()), _dom_classes=('widg…



Erro tolerado:


interactive(children=(FloatSlider(value=0.1, description='erro', min=0.001, step=0.001), Output()), _dom_class…

interactive(children=(Button(description='Processar algoritmo', style=ButtonStyle()), Output()), _dom_classes=…

In [114]:
def processar_teste():
    display(teste)
    
widgets.interact_manual.opts['manual_name'] = 'Testar' # muda texto do botao
interact_manual(processar_teste); # metodo a executar quando pressionar o botao

interactive(children=(Button(description='Testar', style=ButtonStyle()), Output()), _dom_classes=('widget-inte…